In [13]:
import rasterio
from matplotlib import pyplot as plt
from rasterio.mask import mask

bands = [
    "T18SUH_20220110T155631_B02.jp2", #blue
    "T18SUH_20220110T155631_B03.jp2", #green
    "T18SUH_20220110T155631_B04.jp2", #red
]

for band in bands:

    # Open our image
    my_image = rasterio.open(band)
    band1 = my_image.read(1)

    # As an example, we can index into the coordinates:
    xmin = 319677
    ymin = 4295835
    xmax = 323391
    ymax = 4298996

    my_geojson = [{
        "type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
        "coordinates": [ #Here are our actual geometry coordinates
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }]

    # Now we can apply the rasterio mask 
    # We specify the function needs to crop (via crop=True)
    with rasterio.open(image_file) as img:
        clipped, transform = mask(my_image, my_geojson, crop=True)

    # Let's copy the metadata from the original ratserio object
    meta = my_image.meta.copy()

    # We will need to update this metadata, and provide the new clipped boundaries
    meta.update(
        {

            "transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]
        }
    )

    # Note: that here we make the filename the coordinates
    filename_out = '{}_clipped.tif'.format(band) 

    # Finally, we can write new image to a GeoTIFF
    with rasterio.open(filename_out, 'w', **meta) as dst:
        dst.write(clipped)

    print('Writing complete for {}'.format(filename_out))


Writing complete for T18SUH_20220110T155631_B02.jp2_clipped.tif
Writing complete for T18SUH_20220110T155631_B03.jp2_clipped.tif
Writing complete for T18SUH_20220110T155631_B04.jp2_clipped.tif
